In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy # RMSE 계산을 위함
from sklearn.metrics import mean_squared_error, mean_absolute_error # MSE, MAE를 위해 추가

# --- 1. Utility Functions ---

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return 0.0 # 모든 y_true가 0인 경우 MAPE는 0으로 처리
    # 0으로 나누는 경우를 방지하기 위해 작은 epsilon 값을 더할 수도 있습니다.
    # 하지만 여기서는 y_true가 0인 경우를 명시적으로 제외하고 있습니다.
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# --- 2. Data Loading and Preprocessing ---

# 파일 로드
# IMPORTANT: Adjust this path to where your JSON file is located on your local machine.
df = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출 및 SettingWithCopyWarning 해결
df_processed = df[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# user_id와 business_id를 연속적인 정수 ID로 인코딩 (AAT-Rec과의 일관성을 위해 유지)
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

# 데이터 분할 (7:1:2 비율)
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
val_size_ratio = 1 / 8
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

# --- 3. UBCF 모델 구현 및 평가 ---
# Surprise 라이브러리를 위한 데이터 로드
reader = Reader(rating_scale=(1, 5))

# 학습 데이터 (train_df)로 train_set 구성
train_set_for_surprise = train_df[['user_id', 'business_id', 'stars']].copy()
data_for_train = Dataset.load_from_df(train_set_for_surprise, reader)
train_set = data_for_train.build_full_trainset()

# 테스트 데이터 (test_df)로 test_set 구성
test_set = [(row['user_id'], row['business_id'], row['stars']) for _, row in test_df.iterrows()]

# UBCF (User-Based Collaborative Filtering) 모델 정의
ubcf_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True}, k=100)

print("\nUBCF 모델 학습 시작...")
ubcf_model.fit(train_set)
print("UBCF 모델 학습 완료.")

print("UBCF 모델 예측 시작...")
predictions_ubcf = ubcf_model.test(test_set)
print("UBCF 모델 예측 완료.")

# UBCF 성능 지표 계산
true_ratings_ubcf = [pred.r_ui for pred in predictions_ubcf]
predicted_ratings_ubcf = [pred.est for pred in predictions_ubcf]

mse_ubcf = mean_squared_error(true_ratings_ubcf, predicted_ratings_ubcf)
rmse_ubcf = np.sqrt(mse_ubcf) # RMSE는 mse의 제곱근
mae_ubcf = mean_absolute_error(true_ratings_ubcf, predicted_ratings_ubcf)
mape_ubcf = mean_absolute_percentage_error(true_ratings_ubcf, predicted_ratings_ubcf)

print(f"\n--- UBCF (n=100) Performance Metrics ---")
print(f"Mean Squared Error (MSE): {mse_ubcf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_ubcf:.4f}")
print(f"Mean Absolute Error (MAE): {mae_ubcf:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_ubcf:.2f}%")


# --- 4. IBCF 모델 구현 및 평가 ---
ibcf_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': False}, k=100)

print("\nIBCF 모델 학습 시작...")
ibcf_model.fit(train_set) # UBCF와 동일한 train_set 사용
print("IBCF 모델 학습 완료.")

print("IBCF 모델 예측 시작...")
predictions_ibcf = ibcf_model.test(test_set) # UBCF와 동일한 test_set 사용
print("IBCF 모델 예측 완료.")

# IBCF 성능 지표 계산
true_ratings_ibcf = [pred.r_ui for pred in predictions_ibcf]
predicted_ratings_ibcf = [pred.est for pred in predictions_ibcf]

mse_ibcf = mean_squared_error(true_ratings_ibcf, predicted_ratings_ibcf)
rmse_ibcf = np.sqrt(mse_ibcf) # RMSE는 mse의 제곱근
mae_ibcf = mean_absolute_error(true_ratings_ibcf, predicted_ratings_ibcf)
mape_ibcf = mean_absolute_percentage_error(true_ratings_ibcf, predicted_ratings_ibcf)

print(f"\n--- IBCF (n=100) Performance Metrics ---")
print(f"Mean Squared Error (MSE): {mse_ibcf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_ibcf:.4f}")
print(f"Mean Absolute Error (MAE): {mae_ibcf:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_ibcf:.2f}%")

전체 데이터 수: 447796
학습 데이터 수: 313456 (70.00%)
검증 데이터 수: 44780 (10.00%)
테스트 데이터 수: 89560 (20.00%)

UBCF 모델 학습 시작...
Computing the cosine similarity matrix...
Done computing similarity matrix.
UBCF 모델 학습 완료.
UBCF 모델 예측 시작...
UBCF 모델 예측 완료.

--- UBCF (n=100) Performance Metrics ---
Mean Squared Error (MSE): 1.2407
Root Mean Squared Error (RMSE): 1.1139
Mean Absolute Error (MAE): 0.8655
Mean Absolute Percentage Error (MAPE): 35.25%

IBCF 모델 학습 시작...
Computing the cosine similarity matrix...
Done computing similarity matrix.
IBCF 모델 학습 완료.
IBCF 모델 예측 시작...
IBCF 모델 예측 완료.

--- IBCF (n=100) Performance Metrics ---
Mean Squared Error (MSE): 1.3896
Root Mean Squared Error (RMSE): 1.1788
Mean Absolute Error (MAE): 0.8947
Mean Absolute Percentage Error (MAPE): 36.40%


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from surprise import Reader, Dataset, KNNBasic, SVD # SVD 모델을 위해 SVD 임포트
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy # RMSE 계산을 위함
from sklearn.metrics import mean_squared_error, mean_absolute_error # MSE, MAE를 위해 추가

# --- 1. Utility Functions ---

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return 0.0 # 모든 y_true가 0인 경우 MAPE는 0으로 처리
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# --- 2. Data Loading and Preprocessing ---

# 파일 로드
# IMPORTANT: Adjust this path to where your JSON file is located on your local machine.
df = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출 및 SettingWithCopyWarning 해결
df_processed = df[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# user_id와 business_id를 연속적인 정수 ID로 인코딩 (AAT-Rec과의 일관성을 위해 유지)
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

# 데이터 분할 (7:1:2 비율)
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
val_size_ratio = 1 / 8
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

# --- 3. Surprise 라이브러리를 위한 데이터 로드 ---
reader = Reader(rating_scale=(1, 5))

# 학습 데이터 (train_df)로 train_set 구성
train_set_for_surprise = train_df[['user_id', 'business_id', 'stars']].copy()
data_for_train = Dataset.load_from_df(train_set_for_surprise, reader)
train_set = data_for_train.build_full_trainset()

# 테스트 데이터 (test_df)로 test_set 구성
test_set = [(row['user_id'], row['business_id'], row['stars']) for _, row in test_df.iterrows()]




### **UBCF 모델 구현 및 평가**


# UBCF (User-Based Collaborative Filtering) 모델 정의
ubcf_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True}, k=100)

print("\nUBCF 모델 학습 시작...")
ubcf_model.fit(train_set)
print("UBCF 모델 학습 완료.")

print("UBCF 모델 예측 시작...")
predictions_ubcf = ubcf_model.test(test_set)
print("UBCF 모델 예측 완료.")

# UBCF 성능 지표 계산
true_ratings_ubcf = [pred.r_ui for pred in predictions_ubcf]
predicted_ratings_ubcf = [pred.est for pred in predictions_ubcf]

mse_ubcf = mean_squared_error(true_ratings_ubcf, predicted_ratings_ubcf)
rmse_ubcf = np.sqrt(mse_ubcf) # RMSE는 mse의 제곱근
mae_ubcf = mean_absolute_error(true_ratings_ubcf, predicted_ratings_ubcf)
mape_ubcf = mean_absolute_percentage_error(true_ratings_ubcf, predicted_ratings_ubcf)

print(f"\n--- UBCF (n=100) Performance Metrics ---")
print(f"Mean Squared Error (MSE): {mse_ubcf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_ubcf:.4f}")
print(f"Mean Absolute Error (MAE): {mae_ubcf:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_ubcf:.2f}%")

전체 데이터 수: 447796
학습 데이터 수: 313456 (70.00%)
검증 데이터 수: 44780 (10.00%)
테스트 데이터 수: 89560 (20.00%)

UBCF 모델 학습 시작...
Computing the cosine similarity matrix...
Done computing similarity matrix.
UBCF 모델 학습 완료.
UBCF 모델 예측 시작...
UBCF 모델 예측 완료.

--- UBCF (n=100) Performance Metrics ---
Mean Squared Error (MSE): 1.2407
Root Mean Squared Error (RMSE): 1.1139
Mean Absolute Error (MAE): 0.8655
Mean Absolute Percentage Error (MAPE): 35.25%


In [4]:
# SVD (Singular Value Decomposition) 모델 정의 및 최적 파라미터 적용
# 제공된 Best RMSE params: {'n_factors': 1, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.02}
# MAE params도 동일하게 나왔으므로 같은 파라미터 사용
svd_model = SVD(n_factors=1, n_epochs=30, lr_all=0.005, reg_all=0.02, random_state=42)

print("\nSVD 모델 학습 시작 (최적 파라미터 적용)...")
svd_model.fit(train_set)
print("SVD 모델 학습 완료.")

print("SVD 모델 예측 시작...")
predictions_svd = svd_model.test(test_set)
print("SVD 모델 예측 완료.")

# SVD 성능 지표 계산
true_ratings_svd = [pred.r_ui for pred in predictions_svd]
predicted_ratings_svd = [pred.est for pred in predictions_svd]

mse_svd = mean_squared_error(true_ratings_svd, predicted_ratings_svd)
rmse_svd = np.sqrt(mse_svd) # RMSE는 mse의 제곱근
mae_svd = mean_absolute_error(true_ratings_svd, predicted_ratings_svd)
mape_svd = mean_absolute_percentage_error(true_ratings_svd, predicted_ratings_svd)

print(f"\n--- SVD (Optimal Parameters) Performance Metrics ---")
print(f"Best RMSE Params Applied: {{'n_factors': 1, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.02}}")
print(f"Mean Squared Error (MSE): {mse_svd:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_svd:.4f}")
print(f"Mean Absolute Error (MAE): {mae_svd:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_svd:.2f}%")


SVD 모델 학습 시작 (최적 파라미터 적용)...
SVD 모델 학습 완료.
SVD 모델 예측 시작...
SVD 모델 예측 완료.

--- SVD (Optimal Parameters) Performance Metrics ---
Best RMSE Params Applied: {'n_factors': 1, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.02}
Mean Squared Error (MSE): 1.0684
Root Mean Squared Error (RMSE): 1.0336
Mean Absolute Error (MAE): 0.8039
Mean Absolute Percentage Error (MAPE): 32.46%
